<a href="MKaan_Model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


# Data preparation

In [6]:
# Let's us read the input data
dftest = pd.read_csv('test.csv')

dftest = dftest.astype(str)

dftest.head()


,descripcion,CPV
0,"""servicios de actuaciones musicales en las fie...","92312120,92312130,92312140"
1,"""Suministro, con montaje e instalación incluid...",42513000
2,"""Concurso extraordinario para el transporte y ...","60160000,60000000,60100000,60161000"
3,"""Servicio de Agencia de Viajes para la Univers...",63510000
4,"""Servicios auxiliares de las dependencias de l...","98341120,71326000"


In [7]:
# We delete instances without any CPV

dftest = dftest.loc[~dftest["CPV"].str.contains("na")]


In [8]:
# We transform the class column for multilabel purposes and take the first two digits
dfytest = []
for ind,row in dftest.iterrows():
  dfytest.append(list(set([x[0:2] for x in dftest.at[ind,'CPV'].split(",")])))


In [9]:
mlbin = MultiLabelBinarizer()
y_test = mlbin.fit_transform(dfytest)

In [10]:
print(mlbin.classes_)
labels = mlbin.classes_

['03' '09' '14' '15' '16' '18' '19' '22' '24' '30' '31' '32' '33' '34'
 '35' '37' '38' '39' '41' '42' '43' '44' '45' '48' '50' '51' '55' '60'
 '63' '64' '65' '66' '70' '71' '72' '73' '75' '76' '77' '79' '80' '85'
 '90' '92' '98']


In [11]:
y_test.shape

(69868, 45)

In [12]:
X_test = dftest['descripcion']

# Transformers


In [13]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 6.5 MB 34.7 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 596 kB 32.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("MKaan/multilingual-cpv-sector-classifier")

model = AutoModelForSequenceClassification.from_pretrained("MKaan/multilingual-cpv-sector-classifier")

Downloading:   0%|          | 0.00/7.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [15]:
import torch

input = torch.tensor([tokenizer.encode("Servicio para la elaboración de un inventario de elementos de comunicaciones de red de área local y espacios docentes en sedes docentes de Cataluña")])

In [16]:
input = tokenizer("Servicio para la elaboración de un inventario de elementos de comunicaciones de red de área local y espacios docentes en sedes docentes de Cataluña", return_tensors="pt")

In [17]:
outputs = model(**input)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.7655, -2.5429, -1.1729, -3.8723,  6.2260,  3.1868, -2.4583, -3.0234,
         -2.5327, -1.3344, -0.1546, -1.5412,  0.2152, -2.9852, -3.2581, -1.9709,
         -0.5164, -2.9679,  5.3111, -1.3158,  1.6531,  0.9305, -3.3799, -3.2194,
         -1.3017, -3.4318, -4.6082, -0.6071,  1.3534, -0.9763,  1.0156, -1.6118,
          1.4107,  1.1752, -0.5273, -0.5253,  3.4475,  2.2450, -0.1822, -0.7198,
          1.0854,  1.5528,  0.5950, -0.2185, -1.0718]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [18]:
logits = outputs.logits

import torch.nn.functional as F 

sm = F.softmax(logits)
sm = sm.detach().numpy()[0]

print(list(sm))

[0.0026339847, 9.633947e-05, 0.0003791428, 2.5493724e-05, 0.61957586, 0.029659767, 0.000104845225, 5.9585556e-05, 9.732349e-05, 0.00032260251, 0.0010495794, 0.00026232802, 0.0015193208, 6.1904124e-05, 4.711803e-05, 0.00017069961, 0.00073097815, 6.298292e-05, 0.24817684, 0.00032865122, 0.0063989772, 0.0031066379, 4.1715804e-05, 4.8976126e-05, 0.0003333028, 3.9603907e-05, 1.2214397e-05, 0.0006676085, 0.004742011, 0.00046150142, 0.0033825561, 0.0002444299, 0.005021444, 0.0039679823, 0.0007230558, 0.00072452886, 0.038494468, 0.011565763, 0.0010210106, 0.00059642154, 0.003627185, 0.005788006, 0.002221079, 0.0009846805, 0.0004194577]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


In [19]:
clases = ["75","16","3","77","71","79","24","18","41","44","45","80","31","66","15","39","85","55","42","51","72","38","19","43","33","14","37","30","98","9","64","22","65","32","70","92","50","73","35","76","90","48","63","34","60"]

# Evaluation


In [20]:
ypred = []

row = X_test.iloc[0]
input = torch.tensor([tokenizer.encode(row)])
outputs = model(input)
logits = outputs.logits
sm = F.softmax(logits)
sm = sm.detach().numpy()[0]
ypred.append(sm)
#list(sm)

row = X_test.iloc[1]
input = torch.tensor([tokenizer.encode(row)])
outputs = model(input)
logits = outputs.logits
sm = F.softmax(logits)
sm = sm.detach().numpy()[0]
ypred.append(sm)

print(ypred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


[array([2.04879721e-03, 1.34356351e-05, 3.33348144e-04, 2.50175453e-05,
       5.98756829e-04, 2.45891549e-02, 3.39745093e-05, 3.01558048e-05,
       6.55288095e-05, 2.97733131e-05, 8.45248986e-04, 2.97443965e-03,
       1.20474113e-04, 7.22866389e-05, 1.19521919e-05, 1.54238252e-04,
       1.52365386e-03, 6.47527515e-04, 5.75974002e-04, 3.76747776e-05,
       1.72424875e-03, 3.06284128e-05, 3.90547066e-05, 1.05681771e-04,
       2.25884760e-05, 3.98749617e-05, 5.47564938e-04, 3.77350043e-05,
       5.16335433e-03, 2.27069650e-05, 3.49035341e-04, 2.81741726e-04,
       3.50330432e-04, 5.65671595e-04, 4.62350174e-04, 9.52825189e-01,
       1.10632041e-03, 1.01491562e-04, 7.08925436e-05, 6.78412398e-05,
       1.02007696e-04, 1.95111206e-04, 9.54522693e-04, 3.00757019e-05,
       7.24294805e-05], dtype=float32), array([2.2618393e-04, 5.8193051e-04, 1.9909407e-04, 1.1131687e-03,
       5.6213484e-04, 2.8951181e-04, 2.6976105e-03, 1.8521062e-04,
       1.2902126e-03, 5.1839058e-03, 5.31242

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [21]:
import numpy as np
ypred_softmax = []
ypred_sigmoid = []

sigmoid = torch.nn.Sigmoid()

# softmax
for row in X_test:
  input = torch.tensor([tokenizer.encode(row)])
  outputs = model(input)
  logits = outputs.logits
  
  sm = F.softmax(logits)
  sm = sm.detach().numpy()[0]
  predictions = np.zeros(45)
  predictions[np.where(sm >= 0.5)] = 1
  ypred_softmax.append(predictions)

  probs = sigmoid(logits.squeeze().cpu())
  predictions = np.zeros(probs.shape)
  predictions[np.where(probs >= 0.5)] = 1
  ypred_sigmoid.append(predictions)

ypred_softmax = np.array(ypred_softmax)
ypred_sigmoid = np.array(ypred_sigmoid)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


We reorder the columns so they correspond to the ones by the MKaan model (that were in alphabetical order in English).

In [22]:
ypred_softmax = ypred_softmax[:, [2,29,25,14,1,7,22,31,6,27,12,33,24,43,38,26,21,15,8,18,23,9,10,41,36,19,17,44,42,30,32,13,34,4,20,37,0,39,3,5,11,16,40,35,28]]

In [23]:
ypred_sigmoid = ypred_sigmoid[:, [2,29,25,14,1,7,22,31,6,27,12,33,24,43,38,26,21,15,8,18,23,9,10,41,36,19,17,44,42,30,32,13,34,4,20,37,0,39,3,5,11,16,40,35,28]]

In [24]:

ypred_sigmoid

array([[1., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 1., ..., 1., 0., 1.],
       ...,
       [0., 1., 0., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 0., 1.]])

In [25]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import coverage_error
from sklearn.metrics import label_ranking_average_precision_score
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(ytest,y_pred):
    # finally, compute metrics
    y_true = ytest
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    coverage_err = coverage_error(y_test, y_pred)
    label_ranking_average_precision = label_ranking_average_precision_score(y_test, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'coverage_error': coverage_err,
               'label_ranking_average_precision_score': label_ranking_average_precision}
    return metrics

In [26]:
# softmax
from sklearn.metrics import coverage_error
from sklearn.metrics import label_ranking_average_precision_score

print(f'mkaan softmax:')	
print(multi_label_metrics(y_test, ypred_softmax))


mkaan softmax:
{'f1': 0.6530256109509881, 'roc_auc': 0.7837627867536359, 'accuracy': 0.5970258201179367, 'coverage_error': 18.730863914810786, 'label_ranking_average_precision_score': 0.632987355112001}


In [27]:
# sigmoid
from sklearn.metrics import coverage_error
from sklearn.metrics import label_ranking_average_precision_score

print(f'mkaan sigmoid:')	
print(multi_label_metrics(y_test, ypred_sigmoid))

mkaan sigmoid:
{'f1': 0.12899101871837995, 'roc_auc': 0.8085513084401988, 'accuracy': 0.0, 'coverage_error': 17.182644415182917, 'label_ranking_average_precision_score': 0.07113382793028321}
